<img src="http://www.espol.edu.ec/sites/default/files/nuevaespol/logo_negro.png" width="301" height="58" />

#     FACULTAD DE INGENIER&Iacute;A EN ELECTRICIDAD Y COMPUTACI&Oacute;N
##   DIPLOMADO EN CIENCIA DE DATOS PARA LOS NEGOCIOS
###  Laboratorio: Kafka

<h3><strong>Introducci&oacute;n</strong></h3>

En Kafka, la comunicación entre los clientes y los servidores se realiza con un protocolo TCP simple, de alto rendimiento y agnóstico de lenguaje. Este protocolo está versionado y mantiene la compatibilidad con versiones anteriores. Ofrecemos un cliente Java para Kafka, pero los clientes están disponibles en muchos idiomas.

<h3><strong>Procedimiento</strong></h3>

<h3>Paso 1: Descarga Kafka</h3>

Descargue la versión 2.5.0 y descomprímalo, acorde a su sistema operativo: <a href="https://www.apache.org/dyn/closer.cgi?path=/kafka/2.5.0/kafka_2.12-2.5.0.tgz">aqui</a>

<h3>Paso 2: Inicie el Servidor</h3>

Kafka usa ZooKeeper, por lo que primero debe iniciar un servidor ZooKeeper si aún no tiene uno. Puede usar el script de conveniencia empaquetado con kafka para obtener una instancia de ZooKeeper de un solo nodo rápida y sucia.

> bin/zookeeper-server-start.sh config/zookeeper.properties

Ahora inicie el servidor Kafka:

> bin/kafka-server-start.sh config/server.properties

<h3>Paso 3: Creación de topicos</h3>

Creemos un topico llamado "prueba" con una sola partición y solo una réplica:

> bin/kafka-topics.sh --create --bootstrap-server localhost:9092 --replication-factor 1 --partitions 1 --topic test

> bin/kafka-topics.sh --list --bootstrap-server localhost:9092

<h3>Paso 4: Envíe mensajes</h3>

Kafka viene con un cliente de línea de comandos que tomará información de un archivo o de una entrada estándar y la enviará como mensajes al clúster de Kafka. Por defecto, cada línea se enviará como un mensaje separado.

Ejecute el productor y luego escriba algunos mensajes en la consola para enviarlos al servidor.

> bin/kafka-console-producer.sh --bootstrap-server localhost:9092 --topic test

<h3>Paso 5: Inicie un Consumidor</h3>

Kafka también tiene un consumidor de línea de comandos que enviará los mensajes a la salida estándar.

> bin/kafka-console-consumer.sh --bootstrap-server localhost:9092 --topic test --from-beginning

Si tiene cada uno de los comandos anteriores ejecutándose en un terminal diferente, ahora debería poder escribir mensajes en el terminal productor y verlos aparecer en el terminal consumidor.

Todas las herramientas de línea de comando tienen opciones adicionales; ejecutar el comando sin argumentos mostrará información de uso que los documenta con más detalle.

<h3>Paso 6: Configuración con multi-brokers</h3>

Hasta ahora hemos estado corriendo contra un solo broker, pero eso no es divertido. Para Kafka, un único agente es solo un grupo de tamaño uno, por lo que nada cambia mucho más que iniciar algunas instancias más de agente. Pero solo para sentirlo, expandamos nuestro clúster a tres nodos (todavía todos en nuestra máquina local).

> cp config/server.properties config/server-1.properties

> cp config/server.properties config/server-2.properties

Ahora edite estos nuevos archivos y configure las siguientes propiedades:

config/server-1.properties:

    broker.id=1
    listeners=PLAINTEXT://:9093
    log.dirs=/tmp/kafka-logs-1
 
config/server-2.properties:

    broker.id=2
    listeners=PLAINTEXT://:9094
    log.dirs=/tmp/kafka-logs-2

La propiedad broker.id es el nombre único y permanente de cada nodo en el clúster. Tenemos que anular el puerto y el directorio de registro solo porque los estamos ejecutando todos en la misma máquina y queremos evitar que todos los intermediarios intenten registrarse en el mismo puerto o sobrescribir los datos de los demás.

Ya tenemos Zookeeper y nuestro nodo único iniciado, por lo que solo necesitamos iniciar los dos nuevos nodos:

> bin/kafka-server-start.sh config/server-1.properties &
...

> bin/kafka-server-start.sh config/server-2.properties &
...

Ahora cree un nuevo tema con un factor de replicación de tres:

> bin/kafka-topics.sh --create --bootstrap-server localhost:9092 --replication-factor 3 --partitions 1 --topic my-replicated-topic

Bien, pero ahora que tenemos un clúster, ¿cómo podemos saber qué agente está haciendo qué? Para ver eso, ejecute el comando "describe":

> bin/kafka-topics.sh --describe --bootstrap-server localhost:9092 --topic my-replicated-topic

>Topic:my-replicated-topic   PartitionCount:1    ReplicationFactor:3 Configs:
    Topic: my-replicated-topic  Partition: 0    Leader: 1   Replicas: 1,2,0 Isr: 1,2,0

Aquí hay una explicación de la salida. La primera línea ofrece un resumen de todas las particiones, cada línea adicional brinda información sobre una partición. Como solo tenemos una partición para este tema, solo hay una línea.

<strong>"líder"</strong> es el nodo responsable de todas las lecturas y escrituras para la partición dada. Cada nodo será el líder de una parte de las particiones seleccionada al azar.

<strong>"réplicas"</strong> es la lista de nodos que replican el registro de esta partición independientemente de si son el líder o incluso si están actualmente vivos.

<strong>"isr"</strong> es el conjunto de réplicas "sincronizadas". Este es el subconjunto de la lista de réplicas que actualmente está vivo y atrapado por el líder.

# Bibliografía
<ul>  
<li><a href="https://kafka.apache.org">https://kafka.apache.org</a></li>
<li><a href="https://kafka.apache.org/quickstart">https://kafka.apache.org/quickstart</a></li>
</ul>